# CAMEL AI Task Automation: Transitioning from Manual Operations to Autonomous Agent Handling

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
  Join our Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/camel-ai/camel">Github</a> </i> ⭐
</div>


For more detailed usage information, please refer to our [cookbook](https://colab.research.google.com/drive/1lYgArBw7ARVPSpdwgKLYnp_NEXiNDOd-?usp=sharing)


## Table of Contents
1. [The Problem: Manual GUI Operations & ERP Limitations](#1-the-problem)
2. [Introduction to CAMEL AI Framework](#2-introduction-to-camel-ai)
3. [Understanding ChatAgent and RolePlaying](#3-chatagent-and-roleplaying)
4. [Exploring the Toolkits](#4-exploring-the-toolkits)
5. [Step-by-Step Implementation](#5-step-by-step-implementation)
6. [Running the Complete Example](#6-complete-example)
7. [Conclusion](#7-conclusion)

---

## 1. The Problem: Manual GUI Operations & ERP Limitations {#1-the-problem}

### Current State: Why Manual Operations Fall Short

In today's business environment, we often find ourselves stuck in repetitive, time-consuming workflows:

**Common Pain Points:**
- **Research Tasks**: Manually browsing websites, copying information, formatting reports
- **System Integration**: Switching between multiple applications (browsers, PDFs, spreadsheets)
- **Documentation**: Taking screenshots, organizing files, creating reports
- **Quality Assurance**: Repetitive testing of GUI applications

**Traditional ERP/Automation Limitations:**
- **Rigid Workflows**: Pre-defined processes that can't adapt to changing requirements
- **Limited Integration**: Difficulty connecting disparate systems and applications
- **Manual Intervention**: Constant human oversight and decision-making required
- **Maintenance Overhead**: Brittle scripts that break with UI changes

### The Vision: Intelligent Automation

What if we could create AI agents that:
- **Think and Plan**: Understand complex, multi-step tasks
- **Adapt Dynamically**: Handle unexpected situations and UI changes
- **Collaborate**: Multiple specialized agents working together
- **Use Any Tool**: Seamlessly integrate browsers, terminals, GUI automation, and file systems

---

## 2. Introduction to CAMEL AI Framework {#2-introduction-to-camel-ai}

### What is CAMEL AI?

CAMEL (Communicative Agents for "Mind" Exploration of Large Scale Language Model Society) is a framework for building multi-agent systems where AI agents can:

- **Communicate** using role playing and workforce etc methods to commnunicate with each other
- **task automation** leveraing different tools、MCP、agents to  automate tasks
- **Use Tools and MCMP** to interact with the real world
- **Data generation** leveraging self-improve、self-intruct、evoll-instruct、distillation etc methods to generate data





## 3. Understanding ChatAgent and RolePlaying {#3-chatAgent-and-roleplaying}

### The ChatAgent: Your AI Worker

The `ChatAgent` is a core component of the CAMEL framework, designed to manage conversations between an AI agent and users. It supports multiple large language models (LLMs), allows for structured outputs, and integrates memory management, tool calling, and response termination features.

### Key Features:

1. **Model Flexibility**:
   - Supports various LLM backends such as OpenAI.
   - Can switch between or combine multiple models using strategies like round-robin.
   - Handles both synchronous and asynchronous model calls.

2. **Memory Management**:
   - Maintains conversation history using `AgentMemory`.
   - Automatically manages context length based on token limits or message window size.
   - Allows saving/loading of memory from JSON files.

3. **Tool Integration**:
   - Supports internal function tools (e.g., calculators, search APIs).
   - Allows external tools to be defined, which are not processed by the agent but passed through.
   - Records tool call history with inputs, outputs, and metadata.

4. **Structured Output**:
   - Accepts Pydantic model classes to enforce structured responses (e.g., JSON format).
   - Tries to reformat invalid responses to match the desired schema.

5. **Termination Control**:
   - Integrates `ResponseTerminator` objects to control when the agent should stop responding.
   - Supports manual termination via event flags.

6. **Multi-turn Conversations**:
   - Manages complex dialogues with multi-step interactions.
   - Handles streaming responses from models.

7. **MCP Protocol Support**:
   - Can expose itself as an MCP server to integrate with other systems.



It's ideal for applications such as chatbots, intelligent assistants, and automated agents in research or enterprise environments.

In [ ]:


from camel.agents import ChatAgent
from camel.configs import DeepSeekConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

"""
please set the below os environment:
export DEEPSEEK_API_KEY=""
"""

model = ModelFactory.create(
    model_platform=ModelPlatformType.DEEPSEEK,
    model_type=ModelType.DEEPSEEK_CHAT,
    model_config_dict=DeepSeekConfig(temperature=0.2).as_dict(),
)

# Define system message
sys_msg = "You are a helpful assistant."

# Set agent
camel_agent = ChatAgent(system_message=sys_msg, model=model)

user_msg = """How many Rs are there in the word 'strawberry'?"""

# Get response information
response = camel_agent.step(user_msg)
print(response.msgs[0].content)




### RolePlaying: Collaborative Intelligence

The `RolePlaying` class enables two AI agents to simulate a conversation, where one acts as an assistant and the other as a user. It is commonly used for simulating interactions between humans and AI systems.


### Key Features:

**1. Role Simulation:**  
- Simulates conversations between two roles (e.g., "assistant" and "user").
- Each role is managed by a `ChatAgent`, capable of understanding context and generating natural language responses.

**2. Task Specification & Planning:**  
- **Task Specify Agent**: Makes the task prompt more specific or detailed.  
- **Task Planner Agent**: Breaks down complex tasks into multiple steps.

**3. Critic Evaluation (Optional):**  
- A `CriticAgent` or human reviewer can be introduced to evaluate the quality of the conversation.


**4. Multi-Model Support:**  
- Supports various LLMs (e.g., GPT-4o,DeepSeeek).
- Allows unified or separate model configurations for each role.

**5. Memory:**  
- the memory system is a core module used to store and manage conversation history and context information . It plays a critical role in enabling AI agents to maintain coherence


### Example Use Case:
Imagine you're developing a customer service chatbot. You can use `RolePlaying` to simulate a conversation between a "customer" and a "support agent" to test how well the bot handles different queries. A critic (either AI or human) can then assess whether the responses are accurate and helpful.

In short, `RolePlaying` is a powerful  for leveraging AI agents to interact with each other and solve complex problems.

In [ ]:

from colorama import Fore

from camel.societies import RolePlaying
from camel.utils import print_text_animated


def main(model=None, chat_turn_limit=50) -> None:
    task_prompt = "Develop a trading bot for the stock market"
    role_play_session = RolePlaying(
        assistant_role_name="Python Programmer",
        assistant_agent_kwargs=dict(model=model),
        user_role_name="Stock Trader",
        user_agent_kwargs=dict(model=model),
        task_prompt=task_prompt,
        with_task_specify=True,
        task_specify_agent_kwargs=dict(model=model),
    )

    print(
        Fore.GREEN
        + f"AI Assistant sys message:\n{role_play_session.assistant_sys_msg}\n"
    )
    print(
        Fore.BLUE + f"AI User sys message:\n{role_play_session.user_sys_msg}\n"
    )

    print(Fore.YELLOW + f"Original task prompt:\n{task_prompt}\n")
    print(
        Fore.CYAN
        + "Specified task prompt:"
        + f"\n{role_play_session.specified_task_prompt}\n"
    )
    print(Fore.RED + f"Final task prompt:\n{role_play_session.task_prompt}\n")

    n = 0
    input_msg = role_play_session.init_chat()
    while n < chat_turn_limit:
        n += 1
        assistant_response, user_response = role_play_session.step(input_msg)

        if assistant_response.terminated:
            print(
                Fore.GREEN
                + (
                    "AI Assistant terminated. Reason: "
                    f"{assistant_response.info['termination_reasons']}."
                )
            )
            break
        if user_response.terminated:
            print(
                Fore.GREEN
                + (
                    "AI User terminated. "
                    f"Reason: {user_response.info['termination_reasons']}."
                )
            )
            break

        print_text_animated(
            Fore.BLUE + f"AI User:\n\n{user_response.msg.content}\n"
        )
        print_text_animated(
            Fore.GREEN + "AI Assistant:\n\n"
            f"{assistant_response.msg.content}\n"
        )

        if "CAMEL_TASK_DONE" in user_response.msg.content:
            break

        input_msg = assistant_response.msg


if __name__ == "__main__":
    main()

**Real-world Applications:**
- Competitive research and market analysis
- Data collection from multiple sources
- E-commerce price monitoring
- Social media sentiment analysis

### BrowserToolkit Toolkit: Brings the power of web browsers to your agent.

The `BrowserToolkit` enables interaction with any GUI application:

 The toolkit offers various functions like mouse movement, clicks, dragging, scrolling, keyboard input, screenshot taking, etc.

In [14]:
from camel.toolkits import BrowserToolkit
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

models = {
    "browsing": ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4_1,
        model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
    ),
    "planning": ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4_1,
        model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
    ),
}

tools_list = [
    # Web browsing and research tools
    *BrowserToolkit(
        headless=False,  # Visual mode for debugging
        web_agent_model=models["browsing"],
        planning_agent_model=models["planning"],
    ).get_tools(),
]

print(tools_list)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

### PyAutoGUIToolkit: GUI Automation Master

The `PyAutoGUIToolkit` enables interaction with any GUI application:

 The toolkit offers various functions like mouse movement, clicks, dragging, scrolling, keyboard input, screenshot taking, etc.

In [7]:
from camel.toolkits import PyAutoGUIToolkit
tools_list = [    
    # GUI automation tools
    *PyAutoGUIToolkit().get_tools(),
]
print(tools_list)

[<camel.toolkits.function_tool.FunctionTool object at 0x16ab23430>, <camel.toolkits.function_tool.FunctionTool object at 0x16ab239d0>, <camel.toolkits.function_tool.FunctionTool object at 0x16a504b20>, <camel.toolkits.function_tool.FunctionTool object at 0x16ab23220>, <camel.toolkits.function_tool.FunctionTool object at 0x16ab25f30>, <camel.toolkits.function_tool.FunctionTool object at 0x16ab25b70>, <camel.toolkits.function_tool.FunctionTool object at 0x16a45a5c0>, <camel.toolkits.function_tool.FunctionTool object at 0x16ab23c40>, <camel.toolkits.function_tool.FunctionTool object at 0x16ab25c90>]




### TerminalToolkit: Command Line Power

This toolkit provides a set of functions for terminal operations such as searching for files by name or content, executing shell commands, and managing terminal sessions.

In [10]:

from camel.toolkits import TerminalToolkit

workspace_dir="./"

tools_list = [
      # Terminal/command line tools
    *TerminalToolkit(working_dir=workspace_dir).get_tools(),
]
print(tools_list)

[<camel.toolkits.function_tool.FunctionTool object at 0x16a63c100>, <camel.toolkits.function_tool.FunctionTool object at 0x16a63c130>, <camel.toolkits.function_tool.FunctionTool object at 0x16a63cd00>, <camel.toolkits.function_tool.FunctionTool object at 0x16a831990>, <camel.toolkits.function_tool.FunctionTool object at 0x16a7ab580>, <camel.toolkits.function_tool.FunctionTool object at 0x16a7c2230>, <camel.toolkits.function_tool.FunctionTool object at 0x16a63c7c0>]




### FileWriteToolkit: Document Management

 It provides cross-platform (macOS, Linux, Windows) support for writing to various file formats (Markdown, DOCX, PDF, and plaintext),
replacing text in existing files, automatic backups, custom encoding,and enhanced formatting options for specialized formats.


In [11]:
from camel.toolkits import FileWriteToolkit


tools_list = [

    # File management tools
    *FileWriteToolkit(output_dir="./reports").get_tools(),
]

print(tools_list)

---

## 5. Step-by-Step Implementation {#5-step-by-step-implementation}

### Step 1: Environment Setup

In [ ]:
# Install required packages
!pip install camel-ai colorama pyautogui

# Import necessary modules
import os
from typing import List
from colorama import Fore

from camel.configs import ChatGPTConfig
from camel.agents.chat_agent import ToolCallingRecord
from camel.models import ModelFactory
from camel.societies import RolePlaying
from camel.toolkits import (
    BrowserToolkit,
    FileWriteToolkit,
    TerminalToolkit,
    PyAutoGUIToolkit,
)
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated
from camel.logger import get_logger, set_log_file, set_log_level

### Step 2: Configure Models and Logging

In [ ]:
# Set up logging for debugging
logger = get_logger(__name__)
set_log_file("automation_session.log")
set_log_level(level="DEBUG")

# Define workspace directory
base_dir = os.path.dirname(os.path.abspath(__file__))
workspace_dir = os.path.join(os.path.dirname(os.path.dirname(base_dir)), "workspace")

# Configure AI models for different roles
models = {
    "user": ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4_1,
        model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
    ),
    "assistant": ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4_1,
        model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
    ),
    "browsing": ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4_1,
        model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
    ),
    "planning": ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4_1,
        model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
    ),
}

### Step 3: Define the Complex Task

In [ ]:
# Our multi-step automation task
task_prompt = (
    "Please give me a product research report for OWL: https://github.com/camel-ai/owl. "
    "Let me know how to make it become an awesome commercial product. "
    "Then open the PDF by WPS app in my local computer "
    "and then make a screenshot of this PDF"
)

print(f"Task to automate: {task_prompt}")

**Task Breakdown:**
1. **Research Phase**: Browse GitHub repository, analyze code and documentation
2. **Analysis Phase**: Evaluate commercial potential and provide recommendations
3. **Documentation Phase**: Create a comprehensive research report
4. **Integration Phase**: Open the report in WPS Office
5. **Capture Phase**: Take a screenshot for verification

### Step 4: Initialize Toolkits

In [ ]:
# Create specialized toolkits for different capabilities
tools_list = [
    # Web browsing and research tools
    *BrowserToolkit(
        headless=False,  # Visual mode for debugging
        web_agent_model=models["browsing"],
        planning_agent_model=models["planning"],
    ).get_tools(),
    
    # GUI automation tools
    *PyAutoGUIToolkit().get_tools(),
    
    # Terminal/command line tools
    *TerminalToolkit(working_dir=workspace_dir).get_tools(),
    
    # File management tools
    *FileWriteToolkit(output_dir="./reports").get_tools(),
]

print(f"Available tools: {len(tools_list)} tools loaded")
for i, tool in enumerate(tools_list[:5]):  # Show first 5 tools
    print(f"  {i+1}. {tool.get_function_schema()['name']}")

### Step 5: Create the Agent Society

In [ ]:
# Initialize the collaborative agent system
role_play_session = RolePlaying(
    assistant_role_name="Searcher",
    user_role_name="Professor",
    assistant_agent_kwargs=dict(
        model=models["assistant"],
        tools=tools_list,
    ),
    user_agent_kwargs=dict(
        model=models["user"],
    ),
    task_prompt=task_prompt,
    with_task_specify=False,
)

# Display the system configuration
print(Fore.GREEN + f"AI Assistant sys message:\n{role_play_session.assistant_sys_msg}\n")
print(Fore.BLUE + f"AI User sys message:\n{role_play_session.user_sys_msg}\n")
print(Fore.YELLOW + f"Original task prompt:\n{task_prompt}\n")
print(Fore.RED + f"Final task prompt:\n{role_play_session.task_prompt}\n")

---

## 6. Running the Complete Example {#6-complete-example}

### The Main Automation Loop

In [ ]:
def run_automation_session(chat_turn_limit=50):
    """
    Execute the complete automation workflow
    """
    n = 0
    input_msg = role_play_session.init_chat()
    
    print(Fore.CYAN + "🚀 Starting automation session...\n")
    
    while n < chat_turn_limit:
        n += 1
        print(f"\n--- Turn {n} ---")
        
        # Get responses from both agents
        assistant_response, user_response = role_play_session.step(input_msg)

        # Check for termination conditions
        if assistant_response.terminated:
            print(Fore.GREEN + f"✅ Assistant completed task. Reason: {assistant_response.info['termination_reasons']}")
            break
            
        if user_response.terminated:
            print(Fore.GREEN + f"✅ User satisfied with results. Reason: {user_response.info['termination_reasons']}")
            break

        # Display user (Professor) feedback
        print_text_animated(Fore.BLUE + f"👨‍🏫 Professor:\n{user_response.msg.content}\n")

        # Display assistant (Searcher) actions and thoughts
        print_text_animated(Fore.GREEN + "🔍 Searcher:")
        
        # Show tool usage details
        tool_calls: List[ToolCallingRecord] = [
            ToolCallingRecord(**call.as_dict())
            for call in assistant_response.info['tool_calls']
        ]
        
        for func_record in tool_calls:
            print_text_animated(f"🛠️  Tool Used: {func_record}")
            
        print_text_animated(f"💭 Response: {assistant_response.msg.content}\n")

        # Check for task completion
        if "CAMEL_TASK_DONE" in user_response.msg.content:
            print(Fore.GREEN + "🎉 Task completed successfully!")
            break

        input_msg = assistant_response.msg

# Execute the automation
run_automation_session()

---

## 7. Conclusion {#8-conclusion}

### What We've Accomplished

Through this cookbook, we've demonstrated how to:

1. **Transform Manual Workflows**: Convert complex, multi-step manual processes into automated agent by using role-playing or workforce
2. **Integrate Multiple Systems**: Seamlessly connect web browsers, desktop applications, terminals, and file systems
3. **Create Intelligent Collaboration**: Enable AI agents to work together, share context, and adapt to changing requirements



The future of work is not about replacing humans with AI, but about creating powerful partnerships where AI handles routine tasks and humans focus on creativity, strategy, and relationship building. This cookbook provides the foundation for building those partnerships in your organization.

**Ready to automate your workflows? Start with a simple task and watch the magic happen!** 🚀

That's everything: Got questions about 🐫 CAMEL-AI? Join us on [Discord](https://discord.camel-ai.org)! Whether you want to share feedback, explore the latest in multi-agent systems, get support, or connect with others on exciting projects, we’d love to have you in the community! 🤝

Check out some of our other work:
1. 🐫 Creating Your First CAMEL Agent [free Colab](https://docs.camel-ai.org/cookbooks/create_your_first_agent.html)
2.  Graph RAG Cookbook [free Colab](https://colab.research.google.com/drive/1uZKQSuu0qW6ukkuSv9TukLB9bVaS1H0U?usp=sharing)
3. 🧑‍⚖️ Create A Hackathon Judge Committee with Workforce [free Colab](https://colab.research.google.com/drive/18ajYUMfwDx3WyrjHow3EvUMpKQDcrLtr?usp=sharing)
4. 🔥 3 ways to ingest data from websites with Firecrawl & CAMEL [free Colab](https://colab.research.google.com/drive/1lOmM3VmgR1hLwDKdeLGFve_75RFW0R9I?usp=sharing)
5. 🦥 Agentic SFT Data Generation with CAMEL and Meta Models, Fine-Tuned with Unsloth [free Colab](https://colab.research.google.com/drive/1fdBns2QA1XNwF_tsvG3Hc27QGdViHH3b?usp=sharing)
6. 🦥 Agentic SFT Data Generation with CAMEL and Qwen Models, Fine-Tuned with Unsloth [free Colab](https://colab.research.google.com/drive/1sMnWOvdmASEMhsRIOUSAeYuEywby6FRV?usp=sharing)

Thanks from everyone at 🐫 CAMEL-AI


<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on <a href="https://github.com/camel-ai/camel">Github</a> </i>, join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)  ⭐
</div>